# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [19]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):

    """
    This function assesses a person's medical condition based on various input parameters, including gender, age, respiratory rate, oxygen saturation, heart rate,
    systolic bp, body temperature, alertness, inspried oxygen levels, and performance status. It calculates and provides a severity percentile score to indicate
    the severity of the individual's medical condition.

    Examples:
    >>> priest('MALE', 18, 22.0, 0.95, 105.0, 115.0, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity')
    0.22
    >>> priest('FEMALE', 40, 24.0, 0.96, 105.0, 115.0, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity')
    0.15
    >>> priest('Female', 51, 19.0, 0.99, 90.0, 91.0, 38.1, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity')
    0.18
    """

    score = 0
# Sex
    sex = sex.lower()
    if sex == "male":
        score += 1
    elif sex != "female":
        return None

    # Age
    if 16 <= age <= 49:
        score += 0
    elif 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4
    else:
        return None

    # Respiratory rate
    if respiratory_rate < 9:
        score += 3
    elif 9 <= respiratory_rate <= 11:
        score += 1
    elif 12 <= respiratory_rate <= 20:
        score += 0
    elif 21 <= respiratory_rate <= 24:
        score += 2
    elif respiratory_rate > 24:
        score += 3
 # Oxygen saturation
    if oxygen_saturation > 0.95:
        score += 0
    elif 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif 0.92 <= oxygen_saturation <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3

    # Heart rate
    if heart_rate < 41:
        score += 3
    elif 41 <= heart_rate <= 50:
        score += 1
    elif 51 <= heart_rate <= 90:
        score += 0
    elif 91 <= heart_rate <= 110:
        score += 1
    elif 111 <= heart_rate <= 130:
        score += 2
    elif heart_rate > 130:
        score += 3

    # Systolic BP
    if systolic_bp < 91:
        score += 3
    elif 91 <= systolic_bp <= 100:
        score += 2
    elif 101 <= systolic_bp <= 110:
        score += 1
    elif 111 <= systolic_bp <= 219:
        score += 0
    elif systolic_bp > 219:
        score += 3

    # Temperature
    if temperature < 35.1:
        score += 3
    elif 35.1 <= temperature <= 36.0:
        score += 1
    elif 36.1 <= temperature <= 38.0:
        score += 0
    elif 38.1 <= temperature <= 39.0:
        score += 1
    elif temperature > 39.0:
        score += 2

    # Alertness
    alertness = alertness.lower()
    if alertness == "alert":
        score += 0
    elif "confused" in alertness or "not alert" in alertness:
        score += 3
    else:
        return None

    # Inspired Oxygen
    inspired_oxygen = inspired_oxygen.lower()
    if "air" in inspired_oxygen:
        score += 0
    elif "supplemental oxygen" in inspired_oxygen:
        score += 2
    else:
        return None
# Performance Status
    performance_status = performance_status.lower()
    if "unrestricted normal activity" in performance_status:
        score += 0
    elif "limited strenuous activity" in performance_status:
        score += 1
    elif "limited activity, can self-care" in performance_status:
        score += 2
    elif "limited self-care" in performance_status:
        score += 3
    elif "bed/chair bound" in performance_status:
        score += 4
    else:
        return None

    # Translate score to risk percentage
    if score <= 1:
        return 0.01
    elif score <=3:
        return 0.02
    elif score == 4:
        return 0.03
    elif score == 5:
        return 0.09
    elif score == 6:
        return 0.15
    elif score == 7:
        return 0.18
    elif score == 8:
        return 0.22
    elif score == 9:
        return 0.26
    elif score == 10:
        return 0.29
    elif score == 11:
        return 0.34
    elif score == 12:
        return 0.38
    elif score == 13:
        return 0.46
    elif score == 14:
        return 0.47
    elif score == 15:
        return 0.49
    elif score == 16:
        return 0.55
    elif 17 <= score <= 25:
        return 0.59
    elif score >= 26:
      return 0.99

In [20]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('MALE', 18, 22.0, 0.95, 105.0, 115.0, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity')
Expecting:
    0.22
ok
Trying:
    priest('FEMALE', 40, 24.0, 0.96, 105.0, 115.0, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity')
Expecting:
    0.15
ok
Trying:
    priest('Female', 51, 19.0, 0.99, 90.0, 91.0, 38.1, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity')
Expecting:
    0.18
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [21]:
import requests
def find_hospital(age,sex, risk):
    """(int, str, float) -> (str)
    The function processes input parameters, including age, gender, and risk level, and
    provides the name of the hospital as an output in the form of a string.

    >>> find_hospital(18, 'MALE', 0.22)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(40, 'FEMALE', 0.15)
    'Select Specialty Hospital - Northeast Atlanta'

    >>> find_hospital(51, 'Female', 0.18)
    'Select Specialty Hospital - Northeast Atlanta'


    """

    url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
    response = requests.get(url)
    hospital = response.json()['hospital']
    hospital_name = hospital
    if hospital_name == response.json()['hospital']:
        return hospital_name
    else:
      return None

In [22]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(18, 'MALE', 0.22)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(40, 'FEMALE', 0.15)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(51, 'Female', 0.18)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [23]:
import requests
import json

def get_address(hospital_name):
    """
    (str) -> (str)
    The function is designed to accept the name of a hospital as an input and, in response, provides the coressponding address of the hopsital.

    Examples:
        >>> get_address('SOUTHWEST HOSPITAL AND MEDICAL CENTER')
        '501 FAIRBURN ROAD SW'

        >>> get_address('SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA')
        '1821 CLIFTON ROAD NE'

        >>> get_address('SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA')
        '1821 CLIFTON ROAD NE'
    """
    try:
        # Load the JSON file from the provided URL.
        url = 'https://hds5210-data.s3.amazonaws.com/ga_hospitals.json'
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception if there's an issue with the request.

        # Parse the JSON data.
        data = json.loads(response.text)

        # Find the hospital in the JSON data.
        if hospital_name in data:
            hospital_info = data[hospital_name]
            if 'ADDRESS' in hospital_info:
                return hospital_info['ADDRESS']

        # If the hospital or address is not found, return None.
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
import doctest
doctest.run_docstring_examples(get_address, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_address('SOUTHWEST HOSPITAL AND MEDICAL CENTER')
Expecting:
    '501 FAIRBURN ROAD SW'
ok
Trying:
    get_address('SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA')
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_address('SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA')
Expecting:
    '1821 CLIFTON ROAD NE'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [24]:
import requests
import csv
import json

# Define the process_people function to process the data
def process_people(file_location):
    results = {}
    """
      The function accepts a file path as input and, in return, generates a list of dictionaries. Each dictionary includes a unique patient number as the key and
      a set of associated parameters, such as gender, age, breathing rate, oxygen saturation, heart rate, systolic blood pressure, body temperature, inspired oxygen levels,
      performance status, risk level, the hospital's name, and the hospital's address.
      """

    # Load the data from the file
    response = requests.get(file_location)
    data = response.text.split('\n')

    # Parse the pipe-delimited data
    reader = csv.reader(data, delimiter='|')

    # Skip the header row
    next(reader)

    # Process each row
    for row in reader:
        if len(row) != 11:
            continue  # Skip rows with incorrect format

        patient_number, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status = row

        age = int(age)
        breath = float(breath)
        o2sat = float(o2sat)
        heart = float(heart)
        systolic = float(systolic)
        temp = float(temp)

        risk = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)
        hospital_name = find_hospital(age, sex, risk)
        address = get_address(hospital_name.upper())
        results[patient_number]=[sex,age,breath,o2sat,heart,systolic,temp,alertness,inspired,status,risk,hospital_name,address]

    return results

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [25]:
import requests
response = requests.get('https://hds5210-data.s3.amazonaws.com/people_results.json')
result=response.json()

process= process_people("https://hds5210-data.s3.amazonaws.com/people.psv")
for key in result.keys():
  if result[key]!=process[key]:
    print(f"error at{key}")
print("Both dictionary is same")

Both dictionary is same


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---